In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import warnings
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from agents.matchmanager import buildMatchManager
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import RFE
import random


In [2]:
from agents.matchmanager import MatchManager

from itertools import product

from agents import GreedyAgent, AlphaBetaAgent, RandomAgent
from scenarios import scenarioJunction, scenarioJunctionExo, scenarioTest1v1, scenarioTest2v2
from core.const import RED, BLUE
from agents.ml.simple import SimpleMLAgent
import json
import pandas as pd

In [ ]:
board, state = scenarioJunction()
playerRed = SimpleMLAgent(RED, {'scenario': board.name, 'model': 'RandomForestClassifier'})
playerBlue = SimpleMLAgent(BLUE, {'scenario': board.name, 'model': 'RandomForestClassifier'})


In [ ]:
mm = MatchManager(' ', playerRed, playerBlue, board, state, seed=43)
while not mm.end:
    mm.nextStep()

In [3]:
players=['RandomAgent']
#scenarios=['scenarioTest1v1', 'scenarioTest2v2', 'scenarioTest3v1','scenarioJunction', 'scenarioJunctionExo', 'scenarioRoadblock', 'scenarioBridgeHead']
scenarios=['scenarioJunction']
if __name__ == '__main__':
        for s in scenarios:
            vectors=[]
            winners=[]
            df=pd.DataFrame()
            for i in range(2):

                for p in players:
                    for p1 in players:
                        mm = buildMatchManager('', s, p, p1, seed=random.randint(0,10000))
                        #print('Player 1',p)
                        #print('Player 2',p1)
                        #print('SCenario',s)

                        while not mm.end:
                            mm.nextStep()
                            vectors.append(mm.state.vector())
                        old=df.shape[0]
                        df=pd.DataFrame(vectors)
                        new=df.shape[0]
                        for i in range(new-old):
                            winners.append(mm.winner)

            df.columns=mm.state.vectorInfo()
            df['winner']=winners

            #print('ehi'+s)
            df.to_csv(r'../docs/csv/csv'+s+'.csv')
            #print(df)


In [4]:
df

,team_red_2_0,name_red_2_0,index_red_2_0,kind_red_2_0,move_red_2_0,load_red_2_0,hp_red_2_0,hp_max_red_2_0,int_atk_red_2_0,int-def_red_2_0,...,weapon_SM_blue_1_2,weapon_SR_blue_1_2,Scenario,Turn,ActionType,ActionFigureID,ActionFigureName,ActionTeam,Inizialized,winner
0,red,rTank3,0,2,7,1,1,1,6,0,...,0,0,Junction,0,No Action,No Action,No Action,No Action,True,blue
1,red,rTank3,0,2,7,1,1,1,6,0,...,0,0,Junction,0,Move,2,rInf4,red,True,blue
2,red,rTank3,0,2,7,1,1,1,6,0,...,0,0,Junction,0,Move,2,rInf4,red,True,blue
3,red,rTank3,0,2,7,1,1,1,6,0,...,0,0,Junction,0,Move,1,bInf1,blue,True,blue
4,red,rTank3,0,2,7,1,1,1,6,0,...,0,0,Junction,0,AttackRespond,2,rInf4,red,True,blue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,red,rTank3,0,2,7,1,1,1,4,2,...,0,0,Junction,5,Move,0,rTank3,red,True,blue
150,red,rTank3,0,2,7,1,1,1,4,2,...,0,0,Junction,5,Move,0,bAPC1,blue,True,blue
151,red,rTank3,0,2,7,1,1,1,4,2,...,0,0,Junction,5,Move,0,bAPC1,blue,True,blue
152,red,rTank3,0,2,7,1,0,1,4,2,...,0,0,Junction,5,Attack,2,bInf2,blue,True,blue


In [ ]:
mm.state.lastAction.__class__.__name__

In [ ]:
mm.state.lastAction.team

In [5]:
for c in df.columns:
    print(c)

team_red_2_0
name_red_2_0
index_red_2_0
kind_red_2_0
move_red_2_0
load_red_2_0
hp_red_2_0
hp_max_red_2_0
int_atk_red_2_0
int-def_red_2_0
endurance_red_2_0
stat-name_red_2_0
stat-value_red_2_0
bonus_red_2_0
activated_red_2_0
responded_red_2_0
attacked_red_2_0
moved_red_2_0
passed_red_2_0
killed_red_2_0
hit_red_2_0
attacked-by_red_2_0
can-transport_red_2_0
transport-capacity_red_2_0
len-transporting_red_2_0
transported-by_red_2_0
positionX_red_2_0
positionY_red_2_0
positionZ_red_2_0
defense_basic_red_2_0
defense_smoke_red_2_0
defense_antitank_red_2_0
weapon_CA_red_2_0
weapon_AR_red_2_0
weapon_MG_red_2_0
weapon_AT_red_2_0
weapon_MT_red_2_0
weapon_GR_red_2_0
weapon_SM_red_2_0
weapon_SR_red_2_0
team_red_1_1
name_red_1_1
index_red_1_1
kind_red_1_1
move_red_1_1
load_red_1_1
hp_red_1_1
hp_max_red_1_1
int_atk_red_1_1
int-def_red_1_1
endurance_red_1_1
stat-name_red_1_1
stat-value_red_1_1
bonus_red_1_1
activated_red_1_1
responded_red_1_1
attacked_red_1_1
moved_red_1_1
passed_red_1_1
killed_red_1_

In [ ]:
df.sample(1)

In [ ]:
df['positionZ_red_2_0']

In [ ]:
from core import GM
from core.actions import Action
from core.game.board import GameBoard
from core.game.state import GameState
from agents import Agent, GreedyAgent
import numpy as np
from core.const import RED, BLUE
import pandas as pd
import os.path as op

import joblib

In [ ]:
df

In [ ]:
df['color_red_2_0']

In [ ]:
model=joblib.load('C:\\Users\\Nicol\\Documents\\Master\\SecondoProgetto\\newtechnowar-webui\\code\\models\\Junction_RandomForestClassifier.joblib')

In [ ]:
scores=[]
for figure in state.getFiguresCanBeActivated(RED):
    actions = [GM.actionPassFigure(figure)] + \
              GM.buildAttacks(board, state, figure) + \
              GM.buildMovements(board, state, figure)
    for action in actions:
        newState, outcome = GM.activate(board, state, action)

        X = np.array(newState.vector()).reshape(1, -1)
        cols = newState.vectorInfo()
        df = pd.DataFrame(data=X, columns=cols)
        score = model.predict_proba(df)

        scores.append((score, action))

In [ ]:
scores

In [ ]:
def bestActionRandom(scores):
    sorted_multi_list = sorted(scores, key=lambda x: x[0][0][0])
    if (BLUE):
        bestAction=random.choice(sorted_multi_list[-10:])[1]
    else:
        bestAction=random.choice(sorted_multi_list[:10])[1]
    return bestAction
        
    

In [ ]:
bestScore, bestAction = 0.0, None
#print(sorted(scores, reverse=True))

for probs, action in scores:
    print(probs)
    if (BLUE):
        score = probs[0, 0]
        print("ehi",score)
        print(action)
    else:
        score = probs[0, 1]
        print(score)
    if score >= bestScore:
        bestScore, bestAction = score, action

In [ ]:
from scipy.stats import entropy

In [ ]:
len(scores)

In [ ]:
for probs, action in scores:
    print(entropy(probs[0], base=len(scores)))
